In [60]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### RESNET 32

In [61]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [62]:
# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [67]:
#Train the model per the settings specified in the original RESNET paper
# os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
# !python train.py --config configs/cifar/resnet.yaml \
#     model.resnet.depth 32 \
#     train.batch_size 128 \
#     dataset.name CIFAR10 \
#     train.base_lr 0.1 \
#     train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00 \
#     scheduler.epochs 1
pass

In [68]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    train.batch_size 128 \
    dataset.name CIFAR10_RA_2_5 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00 \
    scheduler.epochs 400

[2020-06-20 03:47:33] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_2_5
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 32
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
  

In [69]:
# Resume training with the un-augmented data
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    train.batch_size 128 \
    config.train.checkpoint = /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00400.pth \
    config.train.resume = True \ 
    dataset.name CIFAR10_RA_2_5 \
    train.base_lr 0.1 \       #### SET BASED ON ENDING LR
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume \
    scheduler.epochs 50



usage: train.py [-h] [--config CONFIG] [--resume RESUME]
                [--local_rank LOCAL_RANK]
                ...

positional arguments:
  options

optional arguments:
  -h, --help            show this help message and exit
  --config CONFIG
  --resume RESUME
  --local_rank LOCAL_RANK


In [5]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/checkpoint_00160.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0160

[2020-06-12 01:00:00] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/checkpoint_00160.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 25.66it/s]
[2020-06-12 01:00:04] __main__ INFO: Elapsed 3.08
[2020-06-12 01:00:04] __main__ INFO: Loss 0.4011 Accuracy 0.9232


In [6]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/checkpoint_00100.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0100

[2020-06-12 01:00:11] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/checkpoint_00100.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 26.19it/s]
[2020-06-12 01:00:15] __main__ INFO: Elapsed 3.02
[2020-06-12 01:00:15] __main__ INFO: Loss 0.3604 Accuracy 0.9170


In [2]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    test.batch_size 128 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/checkpoint_00160.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0160_CIFAR101

[2020-06-14 03:30:35] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/checkpoint_00160.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.70it/s]
[2020-06-14 03:30:36] __main__ INFO: Elapsed 0.91
[2020-06-14 03:30:36] __main__ INFO: Loss 0.8051 Accuracy 0.8320


In [10]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    dataset.name CIFAR10_RA_3_5 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/checkpoint_00160.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0160_RANDAUG

[2020-06-16 00:42:33] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/checkpoint_00160.pth
CIFAR 10 Random Augmentation N=3 M=5
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.59it/s]
[2020-06-16 00:42:33] __main__ INFO: Elapsed 0.28
[2020-06-16 00:42:33] __main__ INFO: Loss 6.1237 Accuracy 0.3500


In [3]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['resnet_basic_32', 'resnet_basic_32', 'resnet_basic_32'],
           'Testset': ['cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 160, 160],
           'Loss': [0.3604, 0.4011, 0.8051],
           'Accuracy': [0.9170, 0.9232, 0.8320],
           'Original_Accuracy': [92.5, 92.5, 84.9],
           'Original_CI': [(92.0, 93.0), (92.0, 93.0), (83.2, 86.4)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_32,cifar10,100,0.3604,0.9170,92.5,"(92.0, 93.0)"
1,resnet_basic_32,cifar10,160,0.4011,0.9232,92.5,"(92.0, 93.0)"
2,resnet_basic_32,cifar10.1,160,0.8051,0.8320,84.9,"(83.2, 86.4)"


In [8]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0160/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[ -7.153804  ,  -0.1832159 ,  -0.69570637, ...,  -0.50926757,
         -5.526208  , -12.987257  ],
       [  2.862379  ,   7.963458  ,  -6.603018  , ...,  -4.740323  ,
         25.90399   ,  -0.52988565],
       [  4.25749   ,   8.408992  ,  -4.3299227 , ...,  -2.3715498 ,
         13.468082  ,   4.5792727 ],
       ...,
       [ -4.7270765 ,  -1.2400844 ,   1.3852903 , ...,  -0.51062894,
         -3.399443  ,  -2.4969094 ],
       [ -2.7640457 ,  14.635863  ,   6.7449965 , ...,  -1.3011913 ,
         -3.036379  ,  -7.061736  ],
       [ -2.6933427 ,   1.8961854 ,  -3.6396854 , ...,  18.63456   ,
         -2.7524152 ,  -3.2204888 ]], dtype=float32)

In [9]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/resnet_basic_32'
path = '/home/ec2-user/SageMaker/experiments/resnet_basic_32'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)